Today we will start working with the Fake News Challenge dataset (https://github.com/FakeNewsChallenge/fnc-1). By the end of today, we hope you will be comfortable:
1. Importing and exporting data from a Jupyter notebook
2. Examining the structure of the dataset (how many rows and columns are in the dataset? What does each row and column represent?)
3. With the goal of the Fake News Challenge. How good are you at identifying misleading headlines?  Do you think you can beat an AI?   

Anytime you see ``______ # TODO: FILL IN HERE.`` in the code, you should replace the ``______`` with your own code.

As always, ask your neighbors or an instructor if you have any questions!

# Data Representation and Exploration

### 0. Import the packages we'll need.

In [ ]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os.path

# Adjust settings so that we can fully see the dataset below
pd.set_option('display.max_colwidth', -1)

### 1. Download the data

Go to https://github.com/FakeNewsChallenge/fnc-1 and download the following files:
    - train_stances.csv
    - train_bodies.csv

These files contain the data for the Fake News Challenge in the "csv" format. "csv" stands for "comma-separate values". We'll use this information later, when we tell the program how to load this data.

PRO TIP: Make sure that the downloaded datasets and this jupyter notebook are in the same directory (folder), else you will have problems later.

### 2. Understand what the data contains.

Before we start looking at the data, we should understand what it contains and where it comes from.

Go to http://www.fakenewschallenge.org/ and read up to, and including, the "DATA" section. Try to answer the following questions:

    - What information do these .csv files contain?
    - What is the classification goal posed by the Fake News Challenge?
    - How was this data collected?
    
Discuss your answers with your neighbor or an instructor.

### 3. Load the data.

Fill in the following code to load the data you downloaded into DataFrames.

In [ ]:
train_stances = pd.read_csv("train_stances.csv", encoding = "utf-8")
train_bodies = pd.____("train_bodies.csv", encoding = "utf-8") 

### 4. Look at the layout of the data.

The ``.columns`` parameter of a DataFrame tells us the name of the columns. Run the following cells to examine the column names of the DataFrames we just created.

In [ ]:
train_stances.columns

In [ ]:
train_bodies.____

Notice that each dataset contains the column 'Body ID'. We will use this column later in order to match the datasets.

Now that we've looked at the columns of the dataset, let's look at the rows. How many rows are in each dataset? We can use the ``.shape`` parameter to tell us about the number of rows in each dataset. Can you guess what the second number, returned by ``.shape``, corresponds to?

In [ ]:
train_stances.shape

In [ ]:
train_bodies.shape

Notice that the number of rows in ``train_stances`` is different to the number of rows in ``train_bodies``.  Why is this? How many unique entries are there for the ``Body ID`` variable in each dataset? We can use the ``pd.unique`` function to figure this out, and we can use ``[]`` (square parentheses) to isolate a single column in each dataset:

In [ ]:
len(pd.unique(train_stances['Body ID']))

In [ ]:
len(pd.unique(train_bodies['Body ID']))

Aha! While the ``train_bodies`` dataset has one row for each value of the ``Body ID`` variable, there are multiple rows in ``train_stances`` corresponding to a single value of ``Body ID``.  Let's look at examples in each dataset corresponding to a value of 0 for the ``Body ID``:

In [ ]:
# We expect there to be one row of the train_bodies dataset returned here:
train_bodies[train_bodies['Body ID'] == 0]

In [ ]:
# Let's check that there is exactly one row in train_bodies corresponding to Body ID = 0 by using the .shape parameter 
# discussed before:
train_bodies[train_bodies['Body ID'] == 0].shape

In [ ]:
# We expect there to be multiple rows of the train_stances dataset returned here: (Note: the .head() function 
# really useful for returning just a few rows)
train_stances[train_stances['Body ID'] == 0].____()

In [ ]:
# Let's use the shape parameter once more to check:
train_stances[train_stances['Body ID'] == 0].shape

Hence, the ``train_stances`` dataset contains multiple headlines corresponding to each article in ``train_bodies``, and our task is to train an AI to identify the correct headline associated with each article.

### 5. Re-organize the data.

The data we loaded contains all the information we need, but it puts different pieces of information about the same article in different DataFrames. To make the data easier to work with, we'd like to put the information about each article in one DataFrame.

This function reads a dataset into a single DataFrame.

In [ ]:
train_data = pd.merge(train_bodies, train_stances, on='Body ID')

In [ ]:
# Let's examine the shape of the newly created dataset:
train_data.shape

In [ ]:
# Let's also look at the first few rows of this merged dataset:
train_data.____()

### 6. Exporting the data

Now that we have created an amalgamated dataset, we'd like to export this, so that we can use it in the future:

In [ ]:
train_data.to_csv("train_data.csv", index=False, encoding = "utf-8")

### Extra Challenge
(Only if time permits)

"Extra challenge" sections are a more unguided exploration into the concepts we've discussed. You'll notice less scaffolding for the code -- try implementing these concepts from scratch, and feel free to ask your neighbors or an instructor if you have any questions!

Now let's try looking at some other information about the data. Try to answer the following questions (and any others you think would be useful) -- feel free to look things up online along the way.

    - How many examples are there for each stance? (For instance, how many "unrelated" examples are there?).
        - Are each of the stances equally represented?
    - In general, how long are the headings and article bodies?
        - Does this differ for different stances?
        - How much do these counts vary between different examples?
    - When you read the article and headline and try to decide whether the headline agrees, disagrees, discusses the content of the article or is irrelevant, what factors do you consider?  How can we code these up so that a computer can gain our intuition and perform this classification by itself?

### 7. Stance Variable Distribution

Reminder: the goal of this challenge is to teach the computer to 'read' in article/headline pairs and predict the associated value of the Stance variable (UNRELATED/DISCUSS/AGREE/DISAGREE).  First, we should look at the number of examples that we have for each value of the stance variable that we can use for teaching the computer:

In [ ]:
# Let's look at the distribution of stance variables in the dataset:
train_data.Stance.value_counts()

In [ ]:
# And as a proportion of the rows in the dataset: (use an attribute of train_data that you learned about yesterday to
# divide the count values by the number of rows in the dataset)
train_data.Stance.value_counts()/train_data.____[0] # TODO: FILL IN HERE

Thus, we have a very uneven dataset, with many more examples of UNRELATED article/headline pairs compared to DISCUSS/AGREE/DISAGREE article/headline pairs. We will discuss the consequences of this uneven distribution later.

### 8. Exploring the data - Can we find features that are different across stance groups?

In what follows, we are going to work with a reduced subset of the training dataset, which has an equal number of examples from each Stance category. We will return to working with the full dataset later.

#### 8a. Randomly sample 100 examples from each stance class


In [ ]:
unrelated = train_data[train_data['Stance'] == 'unrelated'].sample(100)

In [ ]:
discuss = train_data[train_data['Stance'] == 'discuss'].sample(_____) # TODO: FILL IN HERE
agree = train_data[train_data['Stance'] == 'agree']._____(_____) # TODO: FILL IN HERE
disagree = train_data[train_data['Stance'] == _____].sample(100) # TODO: FILL IN HERE

In [ ]:
# Check that each class has one hundred samples:
unrelated._____[0] # TODO: FILL IN HERE

#### 8b. What distinguishes each class? 
When headlines and articles are 'unrelated', is the proportion of words from the article contained in the headline different to that for other stance classes?

In [ ]:
# Let's examine the proportion of words that are contained in a headline that are also contained in an article for 
# a headline-article example.  Let's create a function that reads in an example (single row in any of the dataframes we 
# created above) and returns the proportion of the headline that is contained in the article:
def find_headline_in_article_proportion(example):
    # Separate out all of the individual words in headline:
    headline_words = str.split(example['Headline'])
    # Separate out all of the individual words in the article:
    article_words = str.split(example['articleBody'])
    # Counter to count the number of words in the headline that are contained in the article
    counter = 0 
    # Now iterate through each of the words in the headline and check if the word also exists in the article
    for word in headline_words:
        if word in article_words:
            # If the word in the headline is contained in the article, add 1 to the counter
            counter += 1
    # Since the length of each headline is different, what we really want to compare across examples is the proportion 
    # of the headline that is contained in the article 
    proportion = counter/len(headline_words)
    return proportion

In [ ]:
# Let's apply the function to all examples in each of the four dataframes we created above.
# We will save the results to a list:
proportions_unrelated = [] # Save results to this list
for i in range(unrelated.shape[0]):
    # Extract example
    this_example = unrelated.iloc[i]
    # Save result of calling find_headline_in_article_proportion() on example to proportions_unrelated list
    proportions_unrelated.append(find_headline_in_article_proportion(this_example))

In [ ]:
# Now do the same for each of the other stance classes:
# discuss class
proportions_discuss= [] 
for i in range(discuss.shape[0]):
    this_example = discuss.iloc[____] # TODO: FILL IN HERE
    proportions_discuss.append(find_headline_in_article_proportion(_____)) # TODO: FILL IN HERE

In [ ]:
# agree class
proportions_agree= [] 
for i in range(agree.____): # TODO: FILL IN HERE
    this_example = agree.iloc[i]
    proportions_agree.append(find_headline_in_article_proportion(this_example))

In [ ]:
# disagree class
proportions_______= [] # TODO: FILL IN HERE
for i in range(disagree.shape[0]):
    this_example = disagree.iloc[i]
    proportions_disagree.append(_______(this_example)) # TODO: FILL IN HERE

#### 8c. Calculate mean proportion for each stance class.
What do you notice?

In [ ]:
# unrelated
np.mean(proportions_unrelated)

In [ ]:
# agree
np.mean(________agree) # TODO: FILL IN HERE

In [ ]:
# discuss
_____.mean(proportions_discuss) # TODO: FILL IN HERE

In [ ]:
# disagree
np._____(proportions_disagree) # TODO: FILL IN HERE

#### 8d. Visualize the differences in proportions across stance classes
We will create a boxplot to show the differences in proportions across stance classes

In [ ]:
data_to_plot = [proportions_unrelated, proportions_agree, proportions_disagree, ___________] # TODO: FILL IN HERE

In [ ]:
# Create a figure instance
fig = plt.figure(1, figsize=(9, 6))

# Create an axes instance
ax = fig.add_subplot(111)

# Create the boxplot
bp = ax.boxplot(data_to_plot)
# Set the xaxis tick marks
ax.set_xticklabels(['Unrelated', 'Agree', 'Disagree', 'Discuss'])
# Set the yaxis label
ax.set_ylabel('Proportion headline in article')

What do you notice from the boxplot? From the boxplot, could you come up with a rule for the computer to use in order to classify article-headline examples as UNRELATED/AGREE/DISAGREE/DISCUSS? What about if the challenge was only to classify pairs as either UNRELATED or RELATED (with RELATED = AGREE + DISAGREE + DISCUSS)? 